# Indexy
- Index je neměnná sekvence určená k indexaci a řazení
- Může obsahovat duplicitní hodnoty
- Může obsahovat jakýkoliv datový typ
- Může mít více úrovní (jedná se o tzv. multiindex)



## Vybrané metody a atributy pro práci s indexem

### Atributy
- Index.values: Výpis hodnot indexu, viz https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.values.html
- Index.nlevels: Informace o počtu úrovní indexu, viz https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.nlevels.html
- Index.is_unique: Informace o tom, jestli jsou hodnoty indexu unikátní, viz https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.unique.html
- Index.hasnans: Informace o tom, jestli je v indexu hodnota NaN, viz https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.hasnans.html
    
### Metody
- reset_index(): Resetuje aktuální index a namísto něj použije výchozí hodnoty, viz https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html
- set_index(): Použije existující sloupec DataFrame jako index, viz https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html
- sort_index(): Seřadí hodnoty indexu, viz https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_index.html




In [3]:
# Import potrebnych knihoven
import pandas as pd

# Stazeni .csv souboru publikovaneho na github.com
url = 'https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv'
df = pd.read_csv(url,delimiter = ';')
df['DATE'] = pd.to_datetime(df['DATE'])
df.index.is_unique


True

## Multiindex

- DataFrame může mít víceúroňoví index na řádcích i sloupcích
- Víceúrovňové indexi vznikají i při shlukování dat přes více řádků či při sumarizaci dat za pomoci kontingenční tabulky

## Vybrané metody pro práci s multiindexem

- droplevel(): Odstraní vybranou úroveň multiindexu, viz https://pandas.pydata.org/docs/reference/api/pandas.MultiIndex.droplevel.html
- sort_level(): Seřadí vybrou úroveň multiindexu, viz https://pandas.pydata.org/docs/reference/api/pandas.MultiIndex.sortlevel.html
- reorder_levels(): Změní pořadí úrovní multiindexu, viz https://pandas.pydata.org/docs/reference/api/pandas.MultiIndex.reorder_levels.html


In [12]:
srazky = df.copy()
# Nastaveni multiindexu
srazky = srazky.set_index(['DATE','NAME','STATION'])
# Zmena poradi urovni multiindexu
srazky = srazky.reorder_levels(['NAME','STATION','DATE'])
# Odstraneni multiindexu na urovni 1
srazky = srazky.droplevel(1,axis=0)
srazky

PRCP  SNWD  TAVG  TMAX  TMIN
NAME   DATE                                    
PIDING 2006-12-01   0.0   0.0   NaN   4.2   1.0
       2006-12-02   0.0   0.0   NaN   1.8  -1.7
       2006-12-03   0.0   0.0   NaN   5.1  -4.6
       2006-12-04  19.0   0.0   NaN   8.3  -0.9
       2006-12-05   0.0   0.0   NaN  12.0   0.9
...                 ...   ...   ...   ...   ...
TURANY 2020-11-14   0.0   NaN   6.2   NaN   NaN
       2020-11-15   0.0   NaN   6.6   NaN   5.7
       2020-11-16   0.5   NaN   5.9   NaN   NaN
       2020-11-17   0.0   NaN   6.5   NaN   1.7
       2020-11-18   0.0   NaN   7.9  12.7   NaN

[67834 rows x 5 columns]

In [15]:
srazky1  = df.copy()
srazky1['ROK'] = srazky1['DATE'].dt.year
srazky1['MESIC'] = srazky1['DATE'].dt.month
# Agregace dat za pomoci kontingencni tabulky s nastavenim sloupcu jako indexu
srazky1 = srazky1.pivot_table(values = ['PRCP'],index=['ROK','MESIC'],columns='NAME',aggfunc='sum',fill_value='-')
srazky1 = srazky1.droplevel(0,axis=1)
srazky1

NAME         GRAZ  KOSICE  MOSNOV PIDING PRAHA KLEMENTINUM  PRAHA LIBUS  \
ROK  MESIC                                                                
2000 1        9.2    18.9    30.7      -              17.0         29.4   
     2        0.5    36.6    20.8      -              18.3         18.5   
     3       67.6    31.6    31.2      -              81.4         48.5   
     4       18.1    43.4    40.4      -               6.5          2.9   
     5       90.9    57.3    71.7      -              38.7         10.9   
...           ...     ...     ...    ...               ...          ...   
2020 7       82.1    91.0    82.2  175.4                 -         21.5   
     8      227.0    62.8    86.9  257.9                 -        129.4   
     9       79.0    63.5   150.4  112.6                 -         65.3   
     10      38.9   107.6   162.9  155.2                 -         58.4   
     11      13.4     7.9    19.7      -                 -         10.5   

NAME        RUZYNE  SALZBURG  TURANY   WIEN  
ROK  MESIC                                   
2000 1        24.7      51.2    34.0   43.7  
     2        23.9      66.1    22.6   35.1  
     3        42.3     154.2    15.5   80.4  
     4         7.4      30.5     5.1   11.1  
     5        29.1     115.3    33.4   50.4  
...            ...       ...     ...    ...  
2020 7        34.5       0.0    55.3   27.1  
     8       138.0       0.0   104.5   33.5  
     9        58.9       0.0    75.8   91.2  
     10       70.3       8.6    95.0  120.9  
     11        7.3       0.0    20.4    7.1  

[251 rows x 10 columns]